## First, let's install and import all required libraries

In [ ]:
#!pip install -r requirements.txt

In [ ]:
import collections                        #This library adds some extras to the standard python data structures
import folium                             #Great library for plotting data on maps
import json                               #This library allows us to load and handle json files with python
import os                                 #This module is usefull for handling paths and directories
import webbrowser                         #This library allows to handle the webbrowser from python
import time
import datetime
import pandas as pd
from folium.plugins import HeatMap

## Now, let's load and analyze our location history json file 

In [ ]:
data = json.load(open('My_Activity.json', encoding="utf8"))

In [ ]:
data

In [ ]:
type(data)

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df = df.drop(columns=['locationInfos','description','subtitles','details'])
df

In [ ]:
df['text_new'] = df['titleUrl'].str.split('@').str[1]

In [ ]:
df['lat']=df['text_new'].str.split(',').str[0]

In [ ]:
df['lon']=df['text_new'].str.split(',').str[1]

In [ ]:
df=df.dropna()

In [ ]:
df=df.drop(columns=['titleUrl','text_new'])

In [ ]:
df["lat"] = pd.to_numeric(df["lat"], downcast="float")

In [ ]:
df["lon"] = pd.to_numeric(df["lon"], downcast="float")

In [ ]:
df

In [ ]:
df.to_json('maphistory.json', orient='records', lines=True)

In [ ]:
df.to_csv('maphistory.csv')

In [ ]:
df2=df.drop(columns=['header','title','time','products'])

In [ ]:
df2

In [ ]:
df2.to_csv('maphistorycoordinates.csv')

In [ ]:
data2 = [json.loads(line) for line in open('maphistory.json', 'r')]

In [ ]:
data2

## Now we can extract and tranform the data we need to make our map 

In [ ]:
coordinates = collections.defaultdict(int)

In [ ]:
coordinates

In [ ]:
unique_coordinates = (0, 0)

In [ ]:
max_magnitude = 0

In [ ]:
"""Here you transform the coordinates given by google in actual longitude 
and latitude coordinates"""

for i, loc in enumerate(data2):
    # print(i)
    # print(loc)
    if "lat" not in loc or "lon" not in loc:
        continue
    coords = (round(loc["lat"], 6),
                round(loc["lon"], 6))
    # print(coords)

        
    """Here you calculate the magnitude for all coordinates"""
        
    #print(loc["timestampMs"])
    coordinates[coords] += 1 #these are the magnitude values we will need for the coordinates dictionary
    #print(coordinates[coords])
            
    if coordinates[coords] > max_magnitude:
        unique_coordinates = coords
        max_magnitude = coordinates[coords]
        #print(unique_coordinates)
        #print(max_magnitude)

In [ ]:
coordinates

In [ ]:
tilesoptions = ["openstreetmap", "StamenTerrain", "stamentoner",
                "stamenwatercolor", "cartodbpositron", "cartodbdark_matter"]

tiles = tilesoptions[0]
zoom_start = 10
radius = 7
blur = 4
min_opacity = 0.2
max_zoom = 10


In [ ]:
map_data = [(coords[0], coords[1], magnitude)
            for coords, magnitude in coordinates.items()] #coords are keys in the dict. magnitudes are the values

In [ ]:
map_data

In [ ]:
# Generate background map
m = folium.Map(location=unique_coordinates,
                zoom_start=zoom_start,
                tiles=tiles)

# Generate heat map
heatmap = HeatMap(map_data,
                    max_val=max_magnitude,
                    min_opacity=min_opacity,
                    radius=radius,
                    blur=blur,
                    max_zoom=max_zoom)

m.add_child(heatmap)

## We can now save our map as an html file and launch it in the browser 

In [ ]:
output_file = tiles + 'heatmaphistoryJuan.html'

In [ ]:
m.save(output_file)

In [ ]:
webbrowser.open("file://" + os.path.realpath(output_file))